<a href="https://colab.research.google.com/github/otanet/NLP_seminar_20201022/blob/main/AllenNLP%E3%81%AB%E3%82%88%E3%82%8B%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86_(2)_%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AB%E3%82%88%E3%82%8B%E6%84%9F%E6%83%85%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AllenNLPによる自然言語処理 (2): 畳み込みニューラルネットワークによる感情分析

**他のノートブックへのリンク：**

* [AllenNLPによる自然言語処理 (1): bag-of-embeddingsモデルによる文書分類](https://colab.research.google.com/drive/1yquPsCgv7EpNPheqt_Th-j-kK3CN2VWQ?usp=sharing)
* [AllenNLPによる自然言語処理 (3): BERTによる固有表現認識](https://colab.research.google.com/drive/13ga1yYYZkosGZy9ZinAB76blb-8k6yby?usp=sharing)

このノートブックでは、ディープラーニングを用いた自然言語処理のためのツールである[AllenNLP](https://allennlp.org/)を用いて、**畳み込みニューラルネットワーク（CNN）**を用いた**感情分析**のシステムを作成します。

感情分析は、テキストに紐付いた感情を分析する基本的な自然言語処理のタスクの1つです。
アンケート結果の集計やソーシャルメディアの解析などの実用的な用途で、幅広く使用されています。

## 環境のセットアップ

本ノートブックの実行に必要なパッケージをインストールします。

In [ ]:
# MeCabをインストール
!apt-get -qq install mecab swig libmecab-dev mecab-ipadic-utf8
# MeCabのPythonバインディングとAllenNLPをインストール
# （boto3はAllenNLPの依存ライブラリだが、最新バージョンだとエラーになるためバージョンを指定）
!pip install "mecab-python3==0.996.5" "allennlp==1.1.0" "boto3==1.15.0"

Selecting previously unselected package libmecab2:amd64.
(Reading database ... 144611 files and directories currently installed.)
Preparing to unpack .../0-libmecab2_0.996-5_amd64.deb ...
Unpacking libmecab2:amd64 (0.996-5) ...
Selecting previously unselected package libmecab-dev.
Preparing to unpack .../1-libmecab-dev_0.996-5_amd64.deb ...
Unpacking libmecab-dev (0.996-5) ...
Selecting previously unselected package mecab-utils.
Preparing to unpack .../2-mecab-utils_0.996-5_amd64.deb ...
Unpacking mecab-utils (0.996-5) ...
Selecting previously unselected package mecab-jumandic-utf8.
Preparing to unpack .../3-mecab-jumandic-utf8_7.0-20130310-4_all.deb ...
Unpacking mecab-jumandic-utf8 (7.0-20130310-4) ...
Selecting previously unselected package mecab-jumandic.
Preparing to unpack .../4-mecab-jumandic_7.0-20130310-4_all.deb ...
Unpacking mecab-jumandic (7.0-20130310-4) ...
Selecting previously unselected package mecab-ipadic.
Preparing to unpack .../5-mecab-ipadic_2.7.0-20070801+main-1_a

## データセットのセットアップ

本ノートブックでは、[Amazon Customer Reviewsデータセット](https://s3.amazonaws.com/amazon-reviews-pds/readme.html)というAmazonに蓄積されたユーザによる製品のレビューのデータセットを使います。

各レビューにはユーザが指定した1〜5の5段階の評価が数値で付与されています。
本章では1と2を否定的、4と5を肯定的であると考えて、レビューのテキストの内容からユーザが該当する製品に対して肯定的か否定的かを判別する感情分析のモデルの実装を行います。


### データセットのダウンロード

データセットをダウンロードして展開します。

下記のコマンドによって、`data/amazon_reviews`に`amazon_reviews_multilingual_JP_v1_00.tsv`が出力されます。

In [ ]:
# データセットの出力ディレクトリを作成
!mkdir -p data/amazon_reviews
# データセットをダウンロード
!wget -q -O data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv.gz "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz"
# データセットを解凍し、data/amazon_reviewsに展開
!gunzip data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv.gz

### データセットを行区切りJSON形式に変換

データセットをAllenNLPの`TextClassificationJsonReader`で読み込める[行区切りJSON（JSON lines）](https://jsonlines.org/)形式に変換します。
このスクリプトはデータセットを読み込んで8:1:1の割合でそれぞれ訓練用、検証用、テスト用に分割し、保存します。

また、このデータセットはウェブサイトから取得されたものであるため、レビューのテキストにはHTMLタグが含まれています。
HTMLタグは今回のタスクにおいては不要なので、HTMLを解析するライブラリである`BeautifulSoup`を用いて除去します。

In [ ]:
import csv
import json
import os
import random
import sys
import warnings
from bs4 import BeautifulSoup

# csvライブラリのフィールドの最大サイズを変更
csv.field_size_limit(1000000)
# BeautifulSoupの出力する警告を抑制
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

# データセットをファイルから読み込む
data = []
with open("data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv") as f:
    reader = csv.reader(f, delimiter="\t")
    # 1行目はヘッダなので無視する
    next(reader)
    for r in reader:
        # レビューのテキストを取得
        review_body = r[13]
        # レビューのテキストからHTMLタグを除去
        review_body = BeautifulSoup(review_body, "html.parser").get_text()
        # 評価の値を取得
        ratings = int(r[7])
        # 評価が2以下の場合に否定的、4以上の場合に肯定的と扱う
        if ratings <= 2:
            data.append(dict(text=review_body, label="negative"))
        elif ratings >= 4:
            data.append(dict(text=review_body, label="positive"))

# データセットから50,000件をランダムに抽出する
random.seed(1)
random.shuffle(data)
data = data[:50000]

# データセットの80%を訓練データ、10%を検証データ、10%をテストデータとして用いる
split_data = {}
eval_size = int(len(data) * 0.1)
split_data["test"] = data[:eval_size]
split_data["validation"] = data[eval_size:eval_size * 2]
split_data["train"] = data[eval_size * 2:]

# JSON Lines形式でデータセットを書き込む
for fold in ("train", "validation", "test"):
    out_file = os.path.join("data/amazon_reviews", "amazon_reviews_{}.jsonl".format(fold))
    with open(out_file, mode="w") as f:
        for item in split_data[fold]:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

実行が終わると下記の3個のファイルが`data/amazon_reviews`に生成されます。

* **訓練用のデータ**: `amazon_reviews_train.jsonl`
* **検証用のデータ**: `amazon_reviews_validation.jsonl`
* **テスト用のデータ**: `amazon_reviews_test.jsonl`

In [ ]:
!ls data/amazon_reviews/*.jsonl

data/amazon_reviews/amazon_reviews_test.jsonl
data/amazon_reviews/amazon_reviews_train.jsonl
data/amazon_reviews/amazon_reviews_validation.jsonl


## 設定ファイルの作成

下記にCNNを用いた感情分析のモデルの設定ファイルを示します。


In [ ]:
model_config = """{
    "random_seed": 1,
    "pytorch_seed": 1,
    "train_data_path": "data/amazon_reviews/amazon_reviews_train.jsonl",
    "validation_data_path": "data/amazon_reviews/amazon_reviews_validation.jsonl",
    "dataset_reader": {
        "type": "text_classification_json",
        "tokenizer": {
            "type": "mecab"
        },
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        }
    },
    "vocabulary": {},
    "datasets_for_vocab_creation": ["train", "validation"],
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "validation_data_loader": {
        "batch_size": 32,
        "shuffle": false
    },
    "model": {
        "type": "basic_classifier",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 100,
                    "padding_index": 0
                }
            }
        },
        "seq2vec_encoder": {
           "type": "cnn",
           "embedding_dim": 100,
           "ngram_filter_sizes": [2],
           "num_filters": 64
        }
    },
    "trainer": {
        "optimizer": {
            "type": "adam"
        },
        "num_epochs": 10,
        "patience": 3,
        "cuda_device": 0
    }
}"""
with open("amazon_reviews.jsonnet", "w") as f:
  f.write(model_config)

本ノートブックで扱う感情分析は、レビューのテキストを肯定的か否定的かに分類する文書分類の問題であるため、設定ファイルは[文書分類の設定ファイル](https://colab.research.google.com/drive/1yquPsCgv7EpNPheqt_Th-j-kK3CN2VWQ?usp=sharing)とモデルに関する設定以外は同一の内容になっています。

モデルに関する設定を見てみましょう。
分類のモデルとして`BasicClassifier`（`basic_classifier`）を用います。

```json
    "model": {
        "type": "basic_classifier",
```

100次元の単語エンベディングを用いてテキストフィールドエンベダを作成します。


```json
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 100
                }
            }
        },
```

seq2vecエンコーダとして、AllenNLPに実装されているCNNの実装である`CnnEncoder`（`cnn`）を使います。
また`CnnEncoder`には窓幅（`ngram_filter_sizes`）を2（連続した2単語を考慮）、フィルタの数（`num_filters`）を64に設定します。

```json
        "seq2vec_encoder": {
           "type": "cnn",
           "embedding_dim": 100,
           "ngram_filter_sizes": [2],
           "num_filters": 64
        }
    }
```

## モデルの訓練

`allennlp train`コマンドを使ってモデルを訓練します。出力ディレクトリを`exp_amazon_reviews`ディレクトリに指定します。

In [ ]:
# MecabTokenizerのPythonファイルをダウンロード
!wget -q https://dl.dropboxusercontent.com/s/2qeulihmlv8btbg/mecab_tokenizer.py
# 出力ディレクトリが既にあった場合は削除
!rm -rf exp_amazon_reviews
# 訓練を実行
!allennlp train --serialization-dir exp_amazon_reviews --include-package mecab_tokenizer amazon_reviews.jsonnet

2020-10-20 05:23:43.150781: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-20 05:23:44,392 - INFO - transformers.file_utils - TensorFlow version 2.3.0 available.
2020-10-20 05:23:45,711 - INFO - allennlp.common.params - random_seed = 1
2020-10-20 05:23:45,711 - INFO - allennlp.common.params - numpy_seed = 1337
2020-10-20 05:23:45,711 - INFO - allennlp.common.params - pytorch_seed = 1
2020-10-20 05:23:45,757 - INFO - allennlp.common.checks - Pytorch version: 1.6.0+cu101
2020-10-20 05:23:45,759 - INFO - allennlp.common.params - type = default
2020-10-20 05:23:45,760 - INFO - allennlp.common.params - dataset_reader.type = text_classification_json
2020-10-20 05:23:45,760 - INFO - allennlp.common.params - dataset_reader.lazy = False
2020-10-20 05:23:45,760 - INFO - allennlp.common.params - dataset_reader.cache_directory = None
2020-10-20 05:23:45,760 - INFO - allennlp.common.params - dataset_reader.max_instances 

検証データセットでの性能が良かったのは2エポック終了時のモデル（`"best_epoch": 1`）で、検証データセットを92.16% （`"best_validation_accuracy": 0.9216` ）の正解率で分類できたことが分かります。


## 性能の評価

`allennlp evaluate`コマンドを使って、テスト用データセットでの性能の評価を行います。

In [ ]:
 !allennlp evaluate --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz data/amazon_reviews/amazon_reviews_test.jsonl

2020-10-20 05:25:39.069199: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-20 05:25:40,341 - INFO - transformers.file_utils - TensorFlow version 2.3.0 available.
2020-10-20 05:25:41,122 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2020-10-20 05:25:41,122 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmptk66qql5
2020-10-20 05:25:41,532 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmptk66qql5/vocabulary.
2020-10-20 05:25:41,787 - INFO - allennlp.common.checks - Pytorch version: 1.6.0+cu101
2020-10-20 05:25:41,789 - INFO - allennlp.commands.evaluate - Reading evaluation data from data/amazon_reviews/amazon_reviews_test.jsonl
reading instances: 5000it [00:04, 1155.61it/s]
2020-10-20 05:25:46,117 - INFO - allennlp.training.util - Iterating over dataset
accuracy: 0.93, loss: 0.20 ||:

テスト用データセットでの正解率は92.96%になりました。

## 学習したモデルを使う

学習したモデルを使って感情分析を実行します。

入力ファイル`sentiment_analysis_input.json`を作成し、`allennlp predict`コマンドで推論を実行します。

In [ ]:
!echo '{"sentence":"この本は、役に立つし、面白い。"}' > sentiment_analysis_input.json
!allennlp predict --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz sentiment_analysis_input.json

2020-10-20 05:25:54.391405: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-20 05:25:55,683 - INFO - transformers.file_utils - TensorFlow version 2.3.0 available.
2020-10-20 05:25:56,476 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2020-10-20 05:25:56,476 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmpz4w6tibk
2020-10-20 05:25:56,876 - INFO - allennlp.common.params - vocabulary.type = from_instances
2020-10-20 05:25:56,876 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmpz4w6tibk/vocabulary.
2020-10-20 05:25:56,991 - INFO - allennlp.common.params - model.type = basic_classifier
2020-10-20 05:25:56,991 - INFO - allennlp.common.params - model.regularizer = None
2020-10-20 05:25:56,992 - INFO - allennlp.common.params - model.text_field_embedder.type = basic
2020-10-20 05:25:56,992

prediction:からはじまる行を整形した結果を示します。 

正しくpositiveに分類されています。

```json
{
  "logits": [3.387394666671753, -2.742903470993042], 
  "probs": [0.9978287816047668, 0.0021712076850235462], 
  "token_ids": [28, 91, 7, 3, 9574, 15, 3, 270, 4], 
  "label": "positive", 
  "tokens": ["\u3053\u306e", "\u672c", "\u306f", "\u3001", "\u5f79\u306b\u7acb\u3064", "\u3057", "\u3001", "\u9762\u767d\u3044", "\u3002"]
}
```

In [ ]:
!echo '{"sentence":"この本は、役に立たないし、面白くない。"}' > sentiment_analysis_input.json
!allennlp predict --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz sentiment_analysis_input.json

2020-10-20 05:25:59.181067: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-20 05:26:00,445 - INFO - transformers.file_utils - TensorFlow version 2.3.0 available.
2020-10-20 05:26:01,242 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2020-10-20 05:26:01,242 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmp5fgtvnh_
2020-10-20 05:26:01,645 - INFO - allennlp.common.params - vocabulary.type = from_instances
2020-10-20 05:26:01,645 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmp5fgtvnh_/vocabulary.
2020-10-20 05:26:01,764 - INFO - allennlp.common.params - model.type = basic_classifier
2020-10-20 05:26:01,764 - INFO - allennlp.common.params - model.regularizer = None
2020-10-20 05:26:01,765 - INFO - allennlp.common.params - model.text_field_embedder.type = basic
2020-10-20 05:26:01,765

正しくnegative (`"label": "negative"`)となりました。

```json
{
  "logits": [-0.8659035563468933, 1.2352415323257446], 
  "probs": [0.10898558795452118, 0.8910144567489624], 
  "token_ids": [28, 91, 7, 3, 22670, 18, 15, 3, 902, 18, 4], 
  "label": "negative", 
  "tokens": ["\u3053\u306e", "\u672c", "\u306f", "\u3001", "\u5f79\u306b\u7acb\u305f", "\u306a\u3044", "\u3057", "\u3001", "\u9762\u767d\u304f", "\u306a\u3044", "\u3002"]
}
```


In [ ]:
!echo '{"sentence":"この本は、役に立たないけど、面白い。"}' > sentiment_analysis_input.json
!allennlp predict --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz sentiment_analysis_input.json

2020-10-20 05:26:04.055254: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-20 05:26:05,357 - INFO - transformers.file_utils - TensorFlow version 2.3.0 available.
2020-10-20 05:26:06,137 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2020-10-20 05:26:06,137 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmpxdh45u81
2020-10-20 05:26:06,532 - INFO - allennlp.common.params - vocabulary.type = from_instances
2020-10-20 05:26:06,532 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmpxdh45u81/vocabulary.
2020-10-20 05:26:06,653 - INFO - allennlp.common.params - model.type = basic_classifier
2020-10-20 05:26:06,654 - INFO - allennlp.common.params - model.regularizer = None
2020-10-20 05:26:06,654 - INFO - allennlp.common.params - model.text_field_embedder.type = basic
2020-10-20 05:26:06,654

こちらはpositive（`"label": "positive"`）に分類されました。

```json
{
  "logits": [2.9268059730529785, -2.210545778274536], 
  "probs": [0.994161069393158, 0.0058389282785356045], 
  "token_ids": [28, 91, 7, 3, 22670, 18, 116, 3, 270, 4], 
  "label": "positive", 
  "tokens": ["\u3053\u306e", "\u672c", "\u306f", "\u3001", "\u5f79\u306b\u7acb\u305f", "\u306a\u3044", "\u3051\u3069", "\u3001", "\u9762\u767d\u3044", "\u3002"]
}
```

**他のノートブックへのリンク：**

* [AllenNLPによる自然言語処理 (1): bag-of-embeddingsモデルによる文書分類](https://colab.research.google.com/drive/1yquPsCgv7EpNPheqt_Th-j-kK3CN2VWQ?usp=sharing)
* [AllenNLPによる自然言語処理 (3): BERTによる固有表現認識](https://colab.research.google.com/drive/13ga1yYYZkosGZy9ZinAB76blb-8k6yby?usp=sharing)